# Data Analysis for Social Sciences: Lab 5

***UNI:*** sk4819 | ***Date:*** November 23, 2021

In [1]:
#Importing libraries
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt

In [2]:
gss = pd.read_csv("GSS.2006.csv.xls", sep = ',')

In [3]:
gss.head()

,vpsu,vstrat,adults,ballot,dateintv,famgen,form,formwt,gender1,hompop,...,away7,gender14,old14,relate14,relhh14,relhhd14,relsp14,where12,where6,where7
0,1,1957,1,3,316,2,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1957,2,2,630,1,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1957,2,2,314,2,1,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1957,1,1,313,1,2,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1957,3,1,322,2,2,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Run a multiple linear probability model.  Tell me how you think your independent variables will affect your dependent variable.  Interpret your results.  Were your expectations correct?  Why or why not?

***Question: What predicts people's opinion on government's responsibility to proivde/fund healthcare- "On the whole, do you think it should or should not be the government's responsibility to Provide health care for the sick." The options are:***
- 1: Definitely should be
- 2: Probably should be
- 3: Probably should NOT be
- 4: Definitely should NOT be

***Independent variables considered:***
- realrinc: respondent income in constant dollars
- helphlth: How successful do you think the government in America is nowadays in providing health care for the sick
    - scale: very successful to very unsuccessful
- degree: Highest educational degree earned by respondent
- polviews: A seven-point scale for political views that people might hold
    - Scale: arranged from extremely liberal to extremely conservative
- health: Would you say your own health, in general, is excellent, good, fair, or poor?
    - Scale: excellent to poor

I think all the independent variables should have a strong effect on the dependent variable. I believe a respondent's political views (liberal vs conservative) will have the strongest effect on their opinion on whether the government should fund/provide healthcare to the sick. I believe the respondents' income too should have a strong effect on the dependent variable: I expect more people on the higher end of the income scale to disagree with the statement'it is the government's responsibility to provide healthcare for the sick', compared to people on the lower end of the income scale. I believe the respondent's health will also dictate their opinion; I expect more people whose health is poor to  agree that it is the government's responsibility to provide healthcare for the them (the sick), than people who are in excellent health.

In [4]:
sub = gss[["hlthcare","realrinc", "helphlth", "degree", "polviews", "health"]].dropna()

In [5]:
sub.head()

,hlthcare,realrinc,helphlth,degree,polviews,health
7,2.0,1137.000,2.0,2.0,4.0,3.0
15,1.0,46901.250,4.0,4.0,2.0,1.0
22,1.0,6395.625,3.0,0.0,4.0,3.0
23,2.0,18476.250,3.0,1.0,2.0,3.0
27,1.0,15633.750,3.0,2.0,4.0,2.0


#### Recoding healthcare into a dummy variable: 
- 1: Government should provide/fund healthcare 
- 0: Government should NOT provide/fund healthcare 

In [6]:
conditions = [
    (sub['hlthcare'] <= 2) ,
    (sub['hlthcare'] >= 3)]
choices = [1, 0]
sub['hlthcare'] = np.select(conditions, choices, default=np.nan)

In [7]:
pd.crosstab(index=sub["hlthcare"], columns="count")

col_0,count
hlthcare,
0.0,86
1.0,797


In [8]:
sub.corr()

,hlthcare,realrinc,helphlth,degree,polviews,health
hlthcare,1.000000,-0.134094,0.129678,-0.133287,-0.253042,0.092999
realrinc,-0.134094,1.000000,-0.036004,0.350034,0.084374,-0.143496
helphlth,0.129678,-0.036004,1.000000,0.066472,-0.234695,0.021603
degree,-0.133287,0.350034,0.066472,1.000000,-0.065432,-0.199697
polviews,-0.253042,0.084374,-0.234695,-0.065432,1.000000,-0.062114
health,0.092999,-0.143496,0.021603,-0.199697,-0.062114,1.000000


In [9]:
lm1 = smf.ols(formula = 'hlthcare ~ realrinc*degree + health + helphlth + polviews', data = sub).fit()
print (lm1.summary())

# "hlthcare","realrinc", , "degree", "polviews", "partyid"
#NOT: race, sex, age, childs, marital, wrkgovt, 

                            OLS Regression Results                            
Dep. Variable:               hlthcare   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     16.21
Date:                Tue, 23 Nov 2021   Prob (F-statistic):           9.41e-18
Time:                        20:06:34   Log-Likelihood:                -132.95
No. Observations:                 883   AIC:                             279.9
Df Residuals:                     876   BIC:                             313.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.0766      0.058     


***Interpretation***:
- `realrinc`:  On average, holding all other variables constant, every additional unit of income is associated with a decrease in the probability of the respondent being in favor of government provided/funded healthcare by 1.454e-06.
- `degree`: On average, holding all other variables constant, every additional degree is associated with a decrease in the probability of the respondent being in favor of government provided/funded healthcare by 0.0379.
- `realrinc:degree`: ***(I included an interaction term between income and degree as I expect them to be highly correlated)*** On average, holding all other variables constant, every additional degree and additional unit of income are associated with an increase in the probability of the respondent being in favor of government provided/funded healthcare by 3.154e-07. 
- `health`: On average, holding all other variables constant, a deterioration of respondent's health is associated with an increase in the probability of the respondent being in favor of government provided/funded healthcare by 0.0163. 
- `helphlth`: On average, holding all other variables constant, a decline in the respondent's opinion of the government's ability to successfully provide healthcare is associated with an increase in the probability of the respondent being in favor of government provided/funded healthcare by 0.0232. 
- `polviews`: On average, holding all other variables constant, every additional movement of respondent's political ideology from liberal to conservative is associated with a decrease in the probability of the respondent being in favor of government provided/funded healthcare by 0.0497.

#### All my expectations were correct. All variables do have a strong effect on the dependent variable. The respondent's political ideology has the strongest effect on their opinion of government provided/funded healthcare. As expected, the respondent's income is negatively affected with this opinion. However, I was surprised to see a negative relationship betweem degrees earned by respondents and their opinion on government funded/provided healthcare.

___
## 2. Run a multiple (binary) logistic model.  (It can be the same as the above LPM or -- even better -- a new model.)  Tell me how you think your independent variables will affect your dependent variable.  Interpret your results in the logit scale.  Were your expectations correct?  Why or why not?

Analyzing the same question: what affects respondents' opinion on government provided/funded healthcare.

I expect the model to have similar results: I expect all independent variables to have a strong effect on the dependent variable. I also expect respondent's political views (liberal vs conservative) to have the strongest effect on their opinion on whether the government should fund/provide healthcare to the sick. Additionally, I believe the respondents' income too would have a strong effect on the dependent variable: I expect more people on the higher end of the income scale to disagree with the statement'it is the government's responsibility to provide healthcare for the sick', compared to people on the lower end of the income scale. I believe the respondent's health will also dictate their opinion; I expect more people whose health is poor to agree that it is the government's responsibility to provide healthcare for the them (the sick), than people who are in excellent health.

In [10]:
logit1 = sm.formula.logit(formula = 'hlthcare ~ realrinc*degree + health + helphlth + polviews', data = sub).fit()
print (logit1.summary()) 

Optimization terminated successfully.
         Current function value: 0.268824
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               hlthcare   No. Observations:                  883
Model:                          Logit   Df Residuals:                      876
Method:                           MLE   Df Model:                            6
Date:                Tue, 23 Nov 2021   Pseudo R-squ.:                  0.1581
Time:                        20:06:34   Log-Likelihood:                -237.37
converged:                       True   LL-Null:                       -281.96
Covariance Type:            nonrobust   LLR p-value:                 4.487e-17
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.8252      0.809      5.966      0.000       3.240       6.410
realrinc      

***Interpretation***:
- `realrinc`:  On average, holding all other variables constant, for every additional unit of income the log odds of a respondent being in favor of government funded/provided healthcare reduce by 1.844e-05.
- `degree`:  On average, holding all other variables constant, for every additional degree the log odds of a respondent being in favor of government funded/provided healthcare reduce by 0.4791.
- `realrinc:degree`: On average, holding all other variables constant, for every additional degree and every additional unit of income, the log odds of a respondent being in favor of government funded/provided healthcare increase by 5.039e-06. 
- `health`: On average, holding all other variables constant, a deterioration of respondent's health is associated with the log odds of a respondent being in favor of government funded/provided healthcare increasing by  0.2541.
- `helphlth`: On average, holding all other variables constant, a decline in the respondent's opinion of the government's ability to successfully provide healthcare is associated with the log odds of a respondent being in favor of government funded/provided healthcare increasing by 0.2670.
- `polviews`: On average, holding all other variables constant, every additional movement of respondent's political ideology from liberal to conservative is associated with the log odds of the respondent being in favor of government funded/provided healthcare decreasing by 0.6229.

#### Again, all my expectations were correct: all the independent variables have a strong effect on the dependent variable. I was right about the direction of the relationship for all the independent variables with the dependent variable. Here again, degree is negatively associated with the dependent variable. As expected, the respondent's political views have one of the strongest effects on the dependent variable.

___
## 3. Get odds ratios from your logit model in Question 2 and interpret some of them.  

In [11]:
np.exp(logit1.params)

Intercept          124.615251
realrinc             0.999982
degree               0.619368
realrinc:degree      1.000005
health               1.289276
helphlth             1.306031
polviews             0.536391
dtype: float64

- `realrinc`:  On average, holding all other variables constant, for every additional unit of income the odds of a respondent being in favor of government funded/provided healthcare increase by (1- 0.999982) 0.000018%.
- `degree`:  On average, holding all other variables constant, for every additional degree the odds of a respondent being in favor of government funded/provided healthcare increase by (1- 0.619368) 0.380632%.
- `realrinc:degree`: On average, holding all other variables constant, for every additional degree and every additional unit of income, the odds of a respondent being in favor of government funded/provided healthcare increase by 1.000005. 
- `health`: On average, holding all other variables constant, a deterioration of respondent's health is associated with the odds of a respondent being in favor of government funded/provided healthcare increasing by about (1.289276 - 1) 28.9%.
- `helphlth`: On average, holding all other variables constant, a decline in the respondent's opinion of the government's ability to successfully provide healthcare is associated with the odds of a respondent being in favor of government funded/provided healthcare increasing by 1.306031.

___
## 4. Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.  

In [12]:
def logit2prob (logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds) 
    return(prob);

In [13]:
intercept = logit1.params.Intercept
b_inc = logit1.params.realrinc
b_degree = logit1.params.degree
b_health = logit1.params.health
b_helphlth = logit1.params.helphlth
b_polviews = logit1.params.polviews

#### 1. Someone with 5000 dollar income, a bachelor's degree, excellent health, and is extremely liberal:

In [14]:
logits_exh = intercept + (5000*b_inc ) + (5*b_degree) + (1*b_health) + (1*b_polviews)
logit2prob(logits_exh)

0.877499618221072

***Someone with 5000 dollar income, a bachelor's degree, excellent health, and is extremely liberal has a 0.877 probability of being in favor of government funded/provided healthcare.***

#### 2. Someone with 1000 dollar income, a high school degree, poor health, and is slightly conservative:


In [15]:
logits_exh = intercept + (1000*b_inc ) + (1*b_degree) + (5*b_health) + (5*b_polviews)
logit2prob(logits_exh)

0.9229900023520637

***Someone with 1000 dollar income, a high school degree, poor health, and is slightly conservative has a 0.923 probability of being in favor of government funded/provided healthcare.***

#### 3. Someone with 20000 dollar income, a graduate degree, excellent health, and leans moderate:

In [16]:
logits_exh = intercept + (20000*b_inc ) + (4*b_degree) + (1*b_health) + (4*b_polviews)
logit2prob(logits_exh)

0.575129759436155

***Someone with 20000 dollar income, a graduate degree, excellent health, and leans moderate has a 0.575 probability of being in favor of government funded/provided healthcare.***